In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from hyperopt import hp, tpe, fmin, Trials
from tqdm import tqdm

import datetime
import copy
import json

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import granular, processing

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing, SEIRHD_Severity, SEIRHD_Bed

from main.seir.fitting import train_val_split, single_fitting_cycle, get_variable_param_ranges
from main.seir.forecast import get_forecast, create_region_csv, create_all_csvs, write_csv
from main.seir.optimiser import Optimiser


from utils.create_report import create_report
from utils.loss import Loss_Calculator
from utils.smooth_jump import smooth_big_jump_stratified, smooth_big_jump
from viz import plot_forecast, plot_fit

## Load Covid19india Data

In [ ]:
loader = Covid19IndiaLoader()
dataframes = loader.get_covid19india_api_data()

In [ ]:
predictions_dict = {}

## Select Districts to fit on

In [ ]:
districts_to_show = [('Maharashtra', 'Mumbai')]

## Get Variable Params

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 1.5),
    'T_inc': (4, 5),
    'T_inf': (3, 4),
    'T_recov_hq': (50, 70),
    'T_recov_non_o2_beds': (20, 30),
    'T_recov_o2_beds': (35, 50),
    'T_recov_icu': (20, 40),
    'T_recov_ventilator': (50, 60),
    'P_non_oxy': (0, 0.15),
    'P_oxy': (0, 0.15),
    'P_icu': (0, 0.05),
    'P_vent': (0, 0.05),
    'P_fatal': (0, 0.2),
    'E_hosp_ratio': (0, 2),
    'I_hosp_ratio': (0, 1)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

## Perform M1 and M2 fits

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=True, granular_data=False, filename=None, #Data
        model=SEIRHD, variable_param_ranges=None, #Choose Model and Ranges
        train_period=14, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=['hospitalised', 'deceased', 'recovered', 'total_infected'], #Compartments to Apply Loss on 
        smooth_jump=True) #Smoothing
    
    predictions_dict[(state, district)]['m2'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=True, granular_data=False, filename=None, #Data
        model=SEIRHD, variable_param_ranges=None, #Choose Model and Ranges
        train_period=14, val_period=0, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=['hospitalised', 'deceased', 'recovered', 'total_infected'], #Compartments to Apply Loss on 
        smooth_jump=True) #Smoothing
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']
    predictions_dict[(state, district)]['data_last_date'] = predictions_dict[(state, district)]['m2']['data_last_date']

In [ ]:
predictions_dict[(state, district)]['m2']['best_params']

## Sensitivity Analysis

In [ ]:
fig, gridsearch_trials = plot_sensitivity(predictions_dict[('Maharashtra', 'Mumbai')], var_name='I_hosp_ratio', which_fit='m1', 
                                          param_range=np.linspace(0, 1, 201), comp_name='total_infected', aux_comp=None)

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [ ]:
lc = Loss_Calculator()

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### M2 Loss DataFrame

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm2')

## Plot Forecasts

In [ ]:
for region in predictions_dict.keys():
    predictions_dict[region]['forecast'] = plot_forecast(predictions_dict[region], region, both_forecasts=True, error_bars=True)

## Create Report

In [ ]:
for region in predictions_dict.keys():
    create_report(predictions_dict[region])

## Create and Save Output CSV

In [ ]:
df_output = create_all_csvs(predictions_dict, icu_fraction=0.02)

In [ ]:
write_csv(df_output)